In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams

rcParams["figure.max_open_warning"] = False

# Stepper motor results

## Analysis

In [ ]:
from utils.evaluate_data import read_df, get_positions, add_pose_to_df

exp_name = '2021_10_12_doa_stepper'
# experiment where we move laterally by 30cm, with fixed angle
degree=0; distance=-300; motors=True; appendix=''
#degree=0; distance=300; motors=False; appendix="" 

# experiment where we rotate by 360 degress, at fixed distance
#degree=-360; distance=15; motors=True; appendix="_new4"
#degree=360; distance=15; motors=False; appendix="_new";

params = dict(
    distance=distance,
    degree=degree,
    props=False,
    bin_selection=3,
    motors=motors,
    source='mono3000',
    appendix=appendix,
    exp_name=exp_name
)
title = f"motors: {params['motors']}, degree: {params['degree']}"

df, df_pos = read_df(**params)
df.head()

add_pose_to_df(df, df_pos, verbose=True)
#df.head()
df.loc[:, "second"] = (df.timestamp - df.timestamp.iloc[0]) * 1e-3

frequencies_matrix = np.array([*df.loc[:,'frequencies']])
plt.figure()
plt.title('frequency selection over time')
plt.pcolormesh(frequencies_matrix.T)
plt.colorbar()

In [ ]:
# inspect geometry

In [ ]:
from utils.geometry import rotate_mics

mic_positions = df.iloc[0].mic_positions
rot_mic_positions = rotate_mics(mic_positions, orientation_deg=-132)
for mic in range(mic_positions.shape[0]):
    plt.scatter(*mic_positions[mic], color=f"C{mic}", label=f"mic{mic}")
    plt.scatter(*rot_mic_positions[mic], color=f"C{mic}", marker="x")
plt.axis("equal")
plt.legend()

In [ ]:
column_names = ["vx"]#, "vy", "x", "y", "yaw_deg", "yaw_rate_deg"]
fig, axs = plt.subplots(len(column_names), 1, squeeze=False)
fig.set_size_inches(10, 10)
for i, column in enumerate(column_names):
    data = df.loc[:, ["timestamp", column]]
    axs[i, 0].scatter(df.second, data[column])
    axs[i, 0].set_ylabel(column)
axs[i, 0].set_xlabel("time [s]")

In [ ]:
from generate_doa_results import get_orientations, get_offsets

orientations_raw = get_orientations(df, method="raw")
orientations = get_orientations(df, method=degree)

plt.figure()
plt.plot(df.second, orientations_raw, label="raw")
plt.plot(df.second, orientations, label=degree)
plt.grid()
plt.xlabel("time [s]")
plt.ylabel("yaw [deg]")
plt.legend()

positions = get_offsets(df, method=distance)
plt.figure()
plt.title(f"offsets for {distance}")
plt.plot(df.second, positions[:, 0], label="x")
plt.plot(df.second, positions[:, 1], label="y")
plt.grid()
plt.xlabel("time [s]")
plt.ylabel("position [m]")
plt.legend()

In [ ]:
# inspect signals

In [ ]:
from audio_stack.beam_former import BeamFormer, normalize_rows, combine_rows
import progressbar

stft = np.array([*df["signals_f"]])  # n_times x n_mics x n_freqs
n_times, n_mics, n_freqs = stft.shape

freqs = df.iloc[0].frequencies
max_bin = np.argmax(np.sum(stft, axis=(0, 1)))
print("signal frequency:", freqs[max_bin])

fig, ax = plt.subplots()
for i in range(n_mics):
    ax.plot(df.second, np.abs(stft[:, i, max_bin]), color=f"C{i}", label=f"mic{i}")
ax.legend(loc="upper right")
ax.set_xlabel("time [s]")
ax.set_title(title)

fig, ax = plt.subplots()
for i in range(n_mics):
    ax.plot(df.audio_timestamp, np.abs(stft[:, i, max_bin]), color=f"C{i}", label=f"mic{i}")
ax.legend(loc="upper right")
ax.set_xlabel("time [s]")
ax.set_title(title)

In [ ]:
plt.figure()
plt.plot(df.second, (df.audio_timestamp - df.audio_timestamp.iloc[0]) * 1e-3)

# test different mic combinations

In [ ]:
def plot_heatmap(ax):
    extra_kwargs = dict(color="red", lw=2)
    orientations_wrap = orientations.copy()[:n_columns]
    orientations_wrap[orientations_wrap > 360] -= 360
    orientations_wrap[orientations_wrap < 0] += 360
    angles = BeamFormer.theta_scan * 180 / np.pi

    plot_times = df.second.values[:n_columns]
    ax.pcolormesh(plot_times, angles, raw_heatmap)
    ax.plot(plot_times, orientations_wrap, **extra_kwargs)
    ax.set_ylim(0, 360)
    ax.set_xlabel("time [s]")
    ax.set_ylabel("angle [deg]")

In [ ]:
import itertools

combination_method = "sum"
# combination_method = "product" # combination across frequencies, and across time
combination_n = 5

# method = "das"
method = "mvdr"

averaging = "none"
alpha = 0.5
n_columns = 50 #len(df)

angles = BeamFormer.theta_scan

for num in [3]: #[2, 3, 4]:
    #combis = [(1, 2)]
    combis = list(itertools.combinations(range(4), num))
    n_combis = len(combis)

    fig, axs = plt.subplots(1, n_combis, sharey=True, sharex=True, squeeze=False)
    axs = axs.reshape((n_combis,))
    fig.suptitle(title)
    fig.set_size_inches(15, 5)
    axs_i = 0

    for i_combi, mic_selection in enumerate(combis):
        print(f"trying combination {mic_selection} ({i_combi + 1}/{len(combis)})")
        mic_positions = df.iloc[0].mic_positions[mic_selection, :]
        beam_former = BeamFormer(mic_positions=mic_positions)

        R_idx = 0
        R_list = [[]] * combination_n
        R = None

        raw_heatmap = np.zeros((len(angles), n_columns))

        with progressbar.ProgressBar(max_value=n_columns, redirect_stdout=True) as p:
            for i_col, (__, row) in enumerate(df.iterrows()):
                signals_f = row.signals_f[:, [max_bin]]  # n_mics x n_freqs
                signals_f = signals_f[mic_selection, :]
                freqs = row.frequencies[[max_bin]]

                R_new = beam_former.get_correlation(
                    signals_f.T
                )  # n_freqs x n_mics x n_mics
                
                if averaging == "iir":
                    if R is None:
                        R = R_new
                    else:
                        R = alpha * R_new + (1 - alpha) * R
                elif averaging == "ma":
                    R_list[R_idx] = R_new
                    R_idx = (R_idx + 1) % combination_n
                    R = np.mean([R_elem for R_elem in R_list if len(R_elem)], axis=0)
                elif averaging == "none":
                    R = R_new

                if method == "mvdr":
                    raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs)
                elif method == "das":
                    raw_spectrum = beam_former.get_das_spectrum(R, freqs)

                #### RAW
                raw_spectrum = combine_rows(
                    raw_spectrum, combination_method, keepdims=True
                )
                raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")
                raw_heatmap[:, i_col] = raw_spectrum.flatten()
                p.update(i_col)
                
                if i_col >= n_columns - 1:
                    break

        plot_heatmap(axs[axs_i])
        axs[axs_i].set_title(f"mics:{mic_selection}")
        axs_i += 1

    # fig.savefig('figures/{exp_name}_mic_selection.png')

# test different dynamic schemes

In [ ]:
def plot_results(fig, axs):
    extra_kwargs = dict(color="red", lw=2)

    axs = axs.reshape((1, 3))
    fig.set_size_inches(15, 5)

    times_plot = df.second[:n_columns]
    orientations_plot = orientations[:n_columns]

    axs[0, 0].pcolormesh(times_plot, angles, raw_heatmap)
    axs[0, 0].plot(times_plot, orientations_plot, **extra_kwargs)
    axs[0, 0].set_title("raw heatmap")

    axs[0, 1].pcolormesh(times_plot, angles, multi_heatmap)
    # axs[0, 1].plot(times, orientations_plot, **extra_kwargs)
    axs[0, 1].set_title("multi heatmap")

    axs[0, 2].pcolormesh(times_plot, angles, dynamic_heatmap)
    # axs[0, 2].plot(times, orientations_plot, **extra_kwargs)
    axs[0, 2].set_title("dynamic heatmap")

    axs[0, 0].set_ylabel("angle [deg]")
    axs[-1, 0].set_xlabel("time [s]")
    axs[-1, 1].set_xlabel("time [s]")
    axs[-1, 2].set_xlabel("time [s]")
    axs[0, 0].set_ylim(0, 360)

In [ ]:
import itertools

mic_selection = range(4)

mic_positions = df.iloc[0].mic_positions[mic_selection, :]
beam_former = BeamFormer(mic_positions=mic_positions)

averaging = 'iir'
method = "mvdr"
normalization_method = "none"
combination_n = 5
beam_former.init_dynamic_estimate(
    freqs, combination_n, combination_method, normalization_method
)
beam_former.init_multi_estimate(freqs, combination_n)

R_idx = 0
R_list = [[]] * combination_n
R = None

n_columns = len(df)

angles = BeamFormer.theta_scan * 180 / np.pi
raw_heatmap = np.zeros((len(angles), n_columns))
dynamic_heatmap = np.zeros((len(angles), n_columns))
multi_heatmap = np.zeros((len(angles), n_columns))

dynamic_spectrum = None
multi_spectrum = None

with progressbar.ProgressBar(max_value=n_columns, redirect_stdout=True) as p:
    
    fig_mics, ax_mics = plt.subplots()
    for i, (__, row) in enumerate(df.iterrows()):
        signals_f = row.signals_f[:, [max_bin]]  # n_mics x n_freqs
        signals_f = signals_f[mic_selection, :]

        orientation_deg = orientations[i]
        position_m = positions[i]
        timestamp = row.timestamp

        R_new = beam_former.get_correlation(signals_f.T)  # n_freqs x n_mics x n_mics
        if averaging == "iir":
            if R is None:
                R = R_new
            else:
                R = alpha * R_new + (1 - alpha) * R
        elif averaging == "ma":
            R_list[R_idx] = R_new
            R_idx = (R_idx + 1) % combination_n
            R = np.mean([R_elem for R_elem in R_list if len(R_elem)], axis=0)
        elif averaging == "none":
            R = R_new

        if method == "mvdr":
            raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs)
        elif method == "das":
            raw_spectrum = beam_former.get_das_spectrum(R, freqs)
        
        time_sec = (row.audio_timestamp - df.audio_timestamp.iloc[0]) * 1e-3

        if not pd.isna(orientation_deg):
            #### DYNAMIC
            beam_former.add_to_dynamic_estimates(
                raw_spectrum, orientation_deg=orientation_deg
            )
            dynamic_spectrum = beam_former.get_dynamic_estimate()
            dynamic_spectrum = combine_rows(
                dynamic_spectrum, combination_method, keepdims=True
            )
            dynamic_spectrum = normalize_rows(dynamic_spectrum, method="zero_to_one")

            #### MULTI
            beam_former.add_to_multi_estimate(
                signals_f.T, freqs, time_sec, orientation_deg, position=position_m
            )
            multi_spectrum = beam_former.get_multi_estimate(method=method)
            multi_spectrum = combine_rows(
                multi_spectrum, combination_method, keepdims=True
            )
            multi_spectrum = normalize_rows(multi_spectrum, method="zero_to_one")
        else:
            print("not updating dynamic and multi:", i)
            
        if i % 10 == 0:
            mics = beam_former.multi_mic_positions
            ax_mics.scatter(mics[:, 0], mics[:, 1], label=f"time {time_sec:.1f}s")
        if i and i % 10 == 0:
            fig_sigs, ax_sigs = plt.subplots(1, 2)
            fig_sigs.set_size_inches(10, 10)
            R_multi = beam_former.get_multi_R()
            #print(np.linalg.cond(R[0, :, :]))
            phases = np.angle(R_multi[0, :, :])
            assert np.allclose(np.diag(phases), 0)
            for i_sig in range(sigs.shape[1]):
                ax_sigs[0].matshow(phases)
                ax_sigs[1].matshow(np.abs(R_multi[0, :, :]))
                square = np.array([[0, 4, 4, 0, 0], [0, 0, 4, 4, 0]]) - 0.5
                for j in range(combination_n):
                    ax_sigs[0].plot(*square + 4 * j, color='red')
                    ax_sigs[1].plot(*square + 4 * j, color='red')
                ax_sigs[0].plot(*square + 4 * beam_former.index_multi, color='white')
                ax_sigs[1].plot(*square + 4 * beam_former.index_multi, color='white')
                #ax_sigs.scatter(np.angle(sigs[0, int(i_sig*4)+1]), [i], label=f"signal {i_sig:.1f}s")
                #ax_sigs.scatter(np.angle(sigs[0, int(i_sig*4)+2]), [i], label=f"signal {i_sig:.1f}s")
                #ax_sigs.scatter(np.angle(sigs[0, int(i_sig*4)+3]), [i], label=f"signal {i_sig:.1f}s")

        #### RAW
        raw_spectrum = combine_rows(raw_spectrum, combination_method, keepdims=True)
        raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")

        raw_heatmap[:, i] = raw_spectrum.flatten()
        if dynamic_spectrum is not None:
            dynamic_heatmap[:, i] = dynamic_spectrum.flatten()
        if multi_spectrum is not None:
            multi_heatmap[:, i] = multi_spectrum.flatten()

        if i >= n_columns - 1:
            break

        p.update(i)
    ax_mics.axis("equal")
    ax_mics.grid()
    ax_mics.set_title("multi mic positions")
    ax_mics.legend(loc="upper right")

    fig, axs = plt.subplots(1, 3, sharey=True, sharex=True)
    plot_results(fig, axs)
    cuts_seconds = [1, 1.5, 2]
    [ax.axvline(s, color="white", ls=":") for s in cuts_seconds for ax in axs]

    fig, axs = plt.subplots(1, len(cuts_seconds), sharey=True)
    fig.set_size_inches(15, 5)
    for i, s in enumerate(cuts_seconds):
        idx = np.argmin(np.abs(df.second.values - s))
        axs[i].plot(angles, raw_heatmap[:, idx], label="raw")
        axs[i].plot(angles, dynamic_heatmap[:, idx], label="dynamic")
        axs[i].plot(angles, multi_heatmap[:, idx], label="multi")
        axs[i].set_title(f"vertical cut at {s} seconds")
        axs[i].set_xlabel("angle [deg]")
    axs[0].set_ylabel("probability [-]")
    plt.legend()

## Plotting (can start here directly!)

Run generate_doa_results.py to generate results. 

In [ ]:
from audio_stack.beam_former import BeamFormer

angles = BeamFormer.theta_scan * 180 / np.pi

exp_name = "2021_10_12_doa_stepper"

fname = f"results/doa_analysis_{exp_name}.pkl"
df_results = pd.read_pickle(fname)
print("read", fname)

df_raw = df_results.loc[(df_results.exp_name == exp_name)]
keys = ["method", "covariance_averaging", "combination_n", "degree", "motors"]
for values, df_chosen in df_raw.groupby(keys, sort=False):
    title_dict = dict(zip(keys, values))
    assert len(df_chosen) == 1, len(df_chosen)

    row_chosen = df_chosen.iloc[0]

    times = np.arange(row_chosen.raw_heatmap.shape[1])

    fig, axs = plt.subplots(1, 3)
    fig.set_size_inches(15, 5)
    axs[0].pcolormesh(times, angles, row_chosen.raw_heatmap)
    axs[0].set_title("raw heatmap")

    axs[1].pcolormesh(times, angles, row_chosen.multi_heatmap)
    axs[1].set_title("multi heatmap")

    axs[2].pcolormesh(times, angles, row_chosen.dynamic_heatmap)
    axs[2].set_title("dynamic heatmap")

    axs[0].set_ylabel("angle [deg]")
    fig.suptitle(title_dict)